In [1]:
### This python script |1|fetches user submissions and records them 
###                    |2|fetches user contests and record time taken to break their limits
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from time import time

In [6]:
### From what difficulty to start considering problems
threshold = 2000
contests = pd.read_csv('contests.csv',index_col = 'id')

In [7]:
### Read problems csv file
probs = pd.read_csv('problemsWithSolved.csv',index_col=['contestId','index']).copy()

###This set contains all tags in codeforces
all_tags = set()
for prob in probs.index:
    all_tags.update(eval(probs.loc[prob]['tags']))
all_tags.remove('*special')
all_tags = dict((i,0) for i in all_tags)

In [8]:
### Read users csv file
users = pd.read_csv('user2100.csv',index_col='handle')

In [ ]:
### Prepare user_tag DataFrame
user_tag = users.drop(columns=['lastName','lstOnline','firstName','country','rank','maxRank'
                                                       ,'organization','friendOf','city'])
### Tags
for tag in all_tags:
    user_tag[tag] = 0
user_tag['all'] = 0

### Ranks
user_tag['pupil'] = np.nan
user_tag['spe'] = np.nan
user_tag['exp'] = np.nan
user_tag['cm'] = np.nan
user_tag['m'] = np.nan
user_tag['im'] = np.nan
user_tag['gm'] = np.nan
user_tag['igm'] = np.nan
user_tag.head(1)
ranks = (('pupil',1200) , ('spe',1400) , ('exp',1600) , ('cm',1900) , ('m',2100) , ('im',2300) , ('gm',2400) , ('igm',2600))

In [10]:
### Calculating number of problems solved from each rating
for user in users.index:
    user_prob = pd.read_csv(f'Submissions/{user}.csv',index_col=['contestId','problem'])
    for prob in user_prob.index:
        try:
            rate = probs.loc[prob,'rating']
            if rate < threshold:
                continue
            tags = eval(probs.loc[prob,'tags'])
            for tag in tags:
                try:
                    user_tag.loc[user,tag] += 1
                except:
                    pass
        except:
            pass

In [11]:
user_tag.head(5)

,rating,maxRating,regTime,schedules,two pointers,constructive algorithms,geometry,matrices,sortings,math,...,shortest paths,all,pupil,spe,exp,cm,m,im,gm,igm
handle,,,,,,,,,,,,,,,,,,,,,
tourist,4009,4009,1265987288,3,64,205,51,27,103,336,...,50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
jiangly,3831,3904,1547985852,2,152,425,87,45,198,649,...,97,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Radewoosh,3646,3759,1385208270,1,54,218,46,19,101,290,...,46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Benq,3620,3833,1435099979,3,67,287,91,51,136,460,...,54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
jqdai0815,3620,3681,1306208507,1,31,120,34,13,50,204,...,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
### Calculating days taken to reach each rank
for contest in range(2036,0,-1):
    try:
        change = pd.read_csv(f'RatingChanges/{contest}.csv',index_col='handle')
        time = contests.loc[contest,'startTimeSeconds']
        for user in change.index:
            regTime = users.loc[user,'regTime']
            rate = change.loc[user,'newRating']
            for rank in ranks:
                if rate >= rank[1]:
                    user_tag.loc[user,rank[0]] = (time-regTime)/(60*60*24)
    except:
        pass

In [33]:
### I forgot to add 'all' column, it is all problems solved
for user in users.index:
    user_prob = pd.read_csv(f'Submissions/{user}.csv')
    user_tag.loc[user,'all'] = user_prob.index.size

In [38]:
user_tag.head(5)

,rating,maxRating,regTime,schedules,two pointers,constructive algorithms,geometry,matrices,sortings,math,...,shortest paths,all,pupil,spe,exp,cm,m,im,gm,igm
handle,,,,,,,,,,,,,,,,,,,,,
tourist,4009,4009,1265987288,3,64,205,51,27,103,336,...,50,2648.0,13.077685,13.077685,13.077685,82.952685,216.994352,266.036019,347.036019,464.001296
jiangly,3831,3904,1547985852,2,152,425,87,45,198,649,...,97,6323.0,0.000556,0.000556,0.000556,99.104722,138.000556,166.125556,173.104722,303.104722
Radewoosh,3646,3759,1385208270,1,54,218,46,19,101,290,...,46,3631.0,276.080208,276.080208,276.080208,309.146181,451.187847,625.184375,637.184375,888.187847
Benq,3620,3833,1435099979,3,67,287,91,51,136,460,...,54,3520.0,20.678484,20.678484,221.633345,365.737512,522.737512,662.695845,662.695845,792.737512
jqdai0815,3620,3681,1306208507,1,31,120,34,13,50,204,...,30,1583.0,45.470984,45.470984,75.304317,516.144595,934.429317,1244.220984,1316.470984,1350.533484


In [39]:
user_tag.to_csv('user_tag.csv',encoding='utf-8',index=True)